In [1]:
from functions import *

# Usage
folder_path = 'C:/Users/Olivi/Documents/Git/Input'  # Update with the correct folder path
output_path = 'C:/Users/Olivi/Documents/Git/Input/Output'  # Output PNG file path

create_booklet(folder_path, output_path)